### Entrenamiento


In [ ]:
import torch
import torch.nn as nn
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader
import os
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import accuracy_score, f1_score, classification_report


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 2. Función de carga con DATA AUGMENTATION para CINE
def get_dataloaders(data_dir, batch_size=32, is_grayscale=False):
    lista_transformaciones = [transforms.Resize((224, 224))]
    
    if is_grayscale:
        lista_transformaciones.append(transforms.Grayscale(num_output_channels=3))
    
    # Añadimos transformaciones que imitan el cine (sombras, giros, ruido)
    lista_transformaciones.extend([
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(15), # La gente no siempre tiene la cara recta
        transforms.ColorJitter(brightness=0.3, contrast=0.3), # Sombras de película
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    data_transforms = transforms.Compose(lista_transformaciones)
    dataset = datasets.ImageFolder(root=data_dir, transform=data_transforms)
    
    # Dividimos en entrenamiento (80%) y validación (20%) para ver métricas reales
    train_size = int(0.8 * len(dataset))
    val_size = len(dataset) - train_size
    train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)
    
    return train_loader, val_loader, dataset.classes

# 3. Función de Evaluación (Métricas de verdad)
def evaluate_model(model, loader, classes, phase_name):
    model.eval()
    y_true, y_pred = [], []
    
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())

    acc = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred, average='weighted')
    
    print(f"\n--- 📈 MÉTRICAS FINALES: {phase_name} ---")
    print(f"Accuracy: {acc:.4f} | F1-Score: {f1:.4f}")
    print(classification_report(y_true, y_pred, target_names=classes))
    
    # Matriz de Confusión
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8,6))
    sns.heatmap(cm, annot=True, fmt='d', xticklabels=classes, yticklabels=classes, cmap='Blues')
    plt.title(f'Matriz de Confusión - {phase_name}')
    plt.show()

# 4. Función de Entrenamiento Actualizada
def train_model(model, train_loader, val_loader, epochs, learning_rate, phase_name, classes):
    print(f"\n=== Iniciando fase: {phase_name} ===")
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    
    for epoch in range(epochs):
        model.train()
        running_loss, all_preds, all_labels = 0.0, [], []
        
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            
        epoch_acc = accuracy_score(all_labels, all_preds)
        print(f"Epoch {epoch+1}/{epochs} - Loss: {running_loss/len(train_loader):.4f} - Acc: {epoch_acc:.4f}")
    
    # Al final de la fase, evaluar con el set de validación
    evaluate_model(model, val_loader, classes, phase_name)
    return model

# 1. Función para calcular pesos automáticamente
def get_class_weights(dataset):
    import numpy as np
    labels = [sample[1] for sample in dataset.samples]
    classes = np.unique(labels)
    
    # 1. Calculamos los pesos balanceados normales
    weights = compute_class_weight(class_weight='balanced', classes=classes, y=labels)
    
    # 2. SUAVIZADO: Aplicamos la raíz cuadrada (Square Root)
    # Esto reduce la "agresividad" del castigo hacia las clases mayoritarias
    weights = np.sqrt(weights)
    
    # 3. NORMALIZACIÓN (Opcional): Que el peso de "Happy" y "Neutral" no baje de 0.8
    # para que no pierdan tanta importancia
    weights = np.maximum(weights, 0.8)
    
    return torch.tensor(weights, dtype=torch.float).to(device)

# 2. Función de carga de datos (modificada para devolver el dataset completo)
def get_dataloaders(data_dir, batch_size=32, is_grayscale=False):
    lista_transformaciones = [
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(15),
        transforms.ColorJitter(brightness=0.3, contrast=0.3)
    ]
    
    if is_grayscale:
        lista_transformaciones.insert(1, transforms.Grayscale(num_output_channels=3))
    
    lista_transformaciones.extend([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    data_transforms = transforms.Compose(lista_transformaciones)
    full_dataset = datasets.ImageFolder(root=data_dir, transform=data_transforms)
    
    # Calcular pesos antes de dividir
    weights = get_class_weights(full_dataset)
    
    train_size = int(0.8 * len(full_dataset))
    val_size = len(full_dataset) - train_size
    train_dataset, val_dataset = torch.utils.data.random_split(full_dataset, [train_size, val_size])

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    
    return train_loader, val_loader, full_dataset.classes, weights

# 3. Función de entrenamiento con pesaje de pérdida
def train_model_weighted(model, train_loader, val_loader, weights, epochs, lr, phase_name, classes):
    print(f"\n=== Fase: {phase_name} (Usando Pesos de Clase) ===")
    
    # AQUÍ ESTÁ EL CAMBIO: Pasamos los pesos al criterio
    criterion = nn.CrossEntropyLoss(weight=weights, label_smoothing=0.1)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    
    # Scheduler: Baja el learning rate si el modelo deja de mejorar
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=2)

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        preds_list, labels_list = [], []

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels) 
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            preds_list.extend(preds.cpu().numpy())
            labels_list.extend(labels.cpu().numpy())

        epoch_loss = running_loss / len(train_loader)
        epoch_acc = accuracy_score(labels_list, preds_list)
        
        print(f"Epoch {epoch+1}/{epochs} | Loss: {epoch_loss:.4f} | Acc: {epoch_acc:.4f}")
        scheduler.step(epoch_loss)

    return model

# ==========================================
# FLUJO PRINCIPAL
# ==========================================
if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # --- FASE 1: FER2013 ---
    train_f, val_f, cls_f, w_fer = get_dataloaders("./Data/fer2013/train", is_grayscale=True)
    print(f"Pesos calculados para FER: {w_fer}")
    
    model = models.resnet18(pretrained=True)
    model.fc = nn.Linear(model.fc.in_features, len(cls_f))
    model = model.to(device)

    # Entrenamiento
    model = train_model_weighted(model, train_f, val_f, w_fer, 5, 0.001, "FER2013", cls_f)
    
    # --- AGREGADO: LLAMADA A EVALUACIÓN PARA FER2013 ---
    evaluate_model(model, val_f, cls_f, "FER2013 Final")

    # --- FASE 2: AFFECTNET ---
    train_a, val_a, cls_a, w_aff = get_dataloaders("./Data/affectnet/train", is_grayscale=False)
    print(f"Pesos calculados para AffectNet: {w_aff}")
    
    # Cirugía de Red (Cambiamos la salida de 7 a 8 clases)
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, len(cls_a))
    model = model.to(device)

    # Entrenamiento con pesos
    model = train_model_weighted(model, train_a, val_a, w_aff, 10, 0.0001, "AffectNet", cls_a)
    
   
    evaluate_model(model, val_a, cls_a, "AffectNet Final")
    
    # Guardar el modelo final
    torch.save(model.state_dict(), "modelo_pesado_final.pth")
    print("\n¡Todo listo! El modelo ha sido entrenado, evaluado y guardado.")

In [ ]:
import cv2
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image
import numpy as np
from insightface.app import FaceAnalysis
from scipy.spatial.distance import cosine
import math
import json
from collections import deque, Counter

# ==========================================
# 1. GESTOR DE IDENTIDAD ROBUSTO (Centroid Tracking)
# ==========================================
class IdentityManager:
    def __init__(self, threshold=0.55):
        self.face_bank = [] 
        self.id_counter = 0
        self.threshold = threshold

    def get_identity(self, embedding):
        if embedding is None: return "Unknown"
        best_match_idx = -1
        min_dist = float('inf')

        for i, person in enumerate(self.face_bank):
            dist = cosine(person["embedding"], embedding)
            if dist < self.threshold and dist < min_dist:
                min_dist = dist
                best_match_idx = i

        if best_match_idx != -1:
            # Actualización dinámica del rostro (Promedio ponderado)
            old_emb = self.face_bank[best_match_idx]["embedding"]
            count = self.face_bank[best_match_idx]["count"]
            new_average = (old_emb * count + embedding) / (count + 1)
            self.face_bank[best_match_idx]["embedding"] = new_average
            self.face_bank[best_match_idx]["count"] += 1
            return self.face_bank[best_match_idx]["id"]
        else:
            new_id = f"Personaje_{self.id_counter}"
            self.face_bank.append({"id": new_id, "embedding": embedding, "count": 1})
            self.id_counter += 1
            return new_id

# ==========================================
# 2. ANALIZADOR DE PELÍCULAS CON FILTRO TEMPORAL
# ==========================================
class MovieAnalyzer:
    def __init__(self, emotion_model_path):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
        # Cargar Modelo 
        self.emotion_model = models.resnet18()
        self.emotion_model.fc = nn.Linear(self.emotion_model.fc.in_features, 8) 
        self.emotion_model.load_state_dict(torch.load(emotion_model_path, map_location=self.device))
        self.emotion_model.to(self.device).eval()
        
        # Detectores
        self.detector = FaceAnalysis(name='buffalo_l', providers=['CPUExecutionProvider'])
        self.detector.prepare(ctx_id=0, det_size=(640, 640))
        self.identity_manager = IdentityManager(threshold=0.55)
        
        
        self.emotion_buffers = {}
        
        self.history = {
            "metadata": {"video": "", "total_frames": 0},
            "personajes": {},
            "bitacora": [],
            "interacciones": []
        }

        self.transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])

    def predict_emotion(self, face_crop, kps=None):
        if face_crop.size == 0: return "neutral"
        
        img = Image.fromarray(cv2.cvtColor(face_crop, cv2.COLOR_BGR2RGB))
        img = self.transform(img).unsqueeze(0).to(self.device)
        
        with torch.no_grad():
            outputs = self.emotion_model(img)
           
            probs = torch.softmax(outputs, dim=1)
            probs[0][4] *= 1.4  
            probs[0][7] *= 1.2  
            
            confianza, predicted = torch.max(probs, 1)
            
        labels = {0:'anger', 1:'contempt', 2:'disgust', 3:'fear', 4:'happy', 5:'neutral', 6:'sad', 7:'surprise'}
        pred_label = labels[predicted.item()]

        
        if kps is not None:
            boca_izq = kps[3]
            boca_der = kps[4]
            ojo_izq = kps[0]
            ojo_der = kps[1]
            
            dist_boca = np.linalg.norm(boca_izq - boca_der)
            dist_ojos = np.linalg.norm(ojo_izq - ojo_der)
            
            
            ratio_sonrisa = dist_boca / dist_ojos
            
            if ratio_sonrisa > 0.85: 
                
                if pred_label in ['sad', 'anger', 'neutral']:
                    return "happy"

        
        if pred_label == 'happy' and confianza.item() > 0.25:
            return "happy"
        elif confianza.item() < 0.40:
            return "neutral"
            
        return pred_label

    def get_stable_emotion(self, char_id, raw_emotion):
        """Aplica la lógica de promedio de 10 frames (Moda)"""
        if char_id not in self.emotion_buffers:
            self.emotion_buffers[char_id] = deque(maxlen=10)
        
        self.emotion_buffers[char_id].append(raw_emotion)
        
        # La emoción estable es la que más se repite en el buffer (Moda)
        counts = Counter(self.emotion_buffers[char_id])
        stable_emotion = counts.most_common(1)[0][0]
        return stable_emotion

    def is_looking(self, p1, p2):
        p1_nose = p1["kps"][2]
        p2_center = p2["center"]
        v_target = np.array([p2_center[0] - p1_nose[0], p2_center[1] - p1_nose[1]])
        eye_center = (p1["kps"][0] + p1["kps"][1]) / 2
        v_gaze = p1_nose - eye_center
        unit_target = v_target / (np.linalg.norm(v_target) + 1e-6)
        unit_gaze = v_gaze / (np.linalg.norm(v_gaze) + 1e-6)
        angle = math.degrees(math.acos(np.clip(np.dot(unit_target, unit_gaze), -1.0, 1.0)))
        return angle < 35

    def process_video(self, video_path):
        cap = cv2.VideoCapture(video_path)
        fps = cap.get(cv2.CAP_PROP_FPS)
        self.history["metadata"]["video"] = video_path
        
        frame_idx = 0
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret: break
            
            # Procesamos frames intercalados para eficiencia
            if frame_idx % 5 == 0:
                segundo = round(frame_idx / fps, 2)
                faces = self.detector.get(frame)
                current_faces_data = []

                for face in faces:
                    char_id = self.identity_manager.get_identity(face.normed_embedding)
                    
                    # 1. Obtener emoción cruda
                    x1, y1, x2, y2 = face.bbox.astype(int)
                    crop = frame[max(0,y1):y2, max(0,x1):x2]
                    raw_emotion = self.predict_emotion(crop)
                    
                    # 2. ESTABILIZACIÓN: Promedio de 10 frames
                    # emotion = self.get_stable_emotion(char_id, raw_emotion)
                    emotion = self.predict_emotion(crop, kps=face.kps)
                    # Guardar en Bitácora y Estadísticas
                    self.history["bitacora"].append({"segundo": segundo, "id": char_id, "emocion": emotion})
                    
                    if char_id not in self.history["personajes"]:
                        self.history["personajes"][char_id] = {"conteo_emociones": {}}
                    self.history["personajes"][char_id]["conteo_emociones"][emotion] = \
                        self.history["personajes"][char_id]["conteo_emociones"].get(emotion, 0) + 1

                    current_faces_data.append({
                        "id": char_id, "center": ((x1+x2)//2, (y1+y2)//2), 
                        "bbox": (x1, y1, x2, y2), "emotion": emotion, "kps": face.kps
                    })

                # Analizar Interacciones (Miradas)
                for p1 in current_faces_data:
                    # Dibujar
                    cv2.rectangle(frame, (p1["bbox"][0], p1["bbox"][1]), (p1["bbox"][2], p1["bbox"][3]), (0, 255, 0), 2)
                    cv2.putText(frame, f"{p1['id']}: {p1['emotion']}", (p1["bbox"][0], p1["bbox"][1]-10), 
                                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

                    for p2 in current_faces_data:
                        if p1["id"] != p2["id"] and self.is_looking(p1, p2):
                            self.history["interacciones"].append({
                                "segundo": segundo, "de": p1["id"], "a": p2["id"], "emocion": p1["emotion"]
                            })
                            cv2.line(frame, (int(p1["kps"][2][0]), int(p1["kps"][2][1])), p2["center"], (0, 255, 255), 2)

                cv2.imshow('Analisis Estabilizado', frame)
                
            if cv2.waitKey(1) & 0xFF == ord('q'): break
            frame_idx += 1
            
        cap.release()
        cv2.destroyAllWindows()
        return self.history

if __name__ == "__main__":
    # Usa el modelo entrenado con pesos (el que mejor F1-Score tuvo)
    analyzer = MovieAnalyzer("modelo_pesado_final.pth")
    resultados = analyzer.process_video("teaser.mp4")
    
    with open("historia_completa.json", "w") as f:
        json.dump(resultados, f, indent=4)
    print("Análisis finalizado con suavizado temporal.")

In [ ]:
import json
import networkx as nx
import matplotlib.pyplot as plt

def generar_grafo_relaciones(json_path):
    with open(json_path, 'r') as f:
        datos = json.load(f)

    G = nx.DiGraph() # Grafo dirigido (A mira a B)

    # Procesar interacciones
    for inter in datos["interacciones"]:
        u, v = inter["de"], inter["a"]
        emocion = inter["emocion"]
        
        if G.has_edge(u, v):
            G[u][v]['weight'] += 1
            G[u][v]['emociones'].append(emocion)
        else:
            G.add_edge(u, v, weight=1, emociones=[emocion])

    # Dibujar
    plt.figure(figsize=(10, 8))
    pos = nx.spring_layout(G)
    
    weights = [G[u][v]['weight'] * 0.5 for u, v in G.edges()]
    
    nx.draw(G, pos, with_labels=True, node_color='skyblue', 
            node_size=2000, width=weights, edge_color='gray', arrowsize=20)
    
    plt.title("Grafo de Relaciones de Miradas en la Película")
    plt.show()



generar_grafo_relaciones("historia_completa.json")